In [15]:
from btree import NodeKey, Node, BTree
import pickle

class DQKV(BTree):
    def __init__(self, type_, values=None):
        self.type = type_
        super().__init__(10)
    
    def get(self, key):
        value = self.search(self.root, key)
        if value is None:
            raise KeyError('There is no value for the key: "{}"'.format(key))
        return value
    
    def set(self, key, value):
        if value is None:
            raise ValueError('Cannot store value: None')
        if not isinstance(key, self.type):
            raise KeyError('Key must be of type: {}'.format(self.type))
        exists = self.search(self.root, key)
        if exists:
            raise ValueError('Cannot store a duplicate key value')
            
        node = NodeKey(key, value)
        self.insert(node)
        
    def range_query(self, interval, inclusive=False):
        if not isinstance(interval, (list, tuple)) and len(interval) !=2:
            raise ValueError('The range argument is incorrect. Please provide a list or tuple of two values specifying the start and end: [start, end]')
            
        lower, upper = interval
        
        if lower is None:
            return self.less_than(self.root, upper, inclusive=inclusive)
        return self.greater_than(self.root, lower, upper_bound=upper, inclusive=inclusive)
    
    def dump(self, filename):
        filename = filename + '.dqdb'
        with open(filename, 'wb') as f:
            pickle.dump(self, f)
            return True
        return False

    @staticmethod
    def load(filename):
        filename = filename + '.dqdb'
        with open(filename, 'rb') as f:
            return pickle.load(f)
        
    def load_from_dict(self, dict):
        for key, value in dict.items():
            self.set(key, value)

In [26]:
dq = DQKV(int)
dq.set(1, 'a')
dq.set(2, 'b')
dq.set(3, 'c')
dq.set(4, 'd')
print(dq.range_query([1,4], inclusive=True))

dq.dump('test')
dqkv = DQKV.load('test')

print(dq.range_query([1,4]))
additional_keys = {
    5: 'e',
    6: 'test',
    7: 'hellow world'
}
dqkv.load_from_dict(additional_keys)
print(dqkv.range_query([3,6], inclusive=True))
print(dqkv.get(7))

[<NodeKey: (1, a)>, <NodeKey: (2, b)>, <NodeKey: (3, c)>, <NodeKey: (4, d)>]
[<NodeKey: (2, b)>, <NodeKey: (3, c)>]
[<NodeKey: (3, c)>, <NodeKey: (4, d)>, <NodeKey: (5, e)>, <NodeKey: (6, test)>]
hellow world
